<a href="https://colab.research.google.com/github/bhargav-borah/Diabetes-Prediction-Model-/blob/main/xg_boost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# XGBoost Classification

## Importing the libraries

In [32]:
import numpy as np
import pandas as pd

## Importing the data set and Data Exploration

In [33]:
dataset = pd.read_csv('diabetes.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

In [34]:
dataset.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [35]:
dataset.shape

(768, 9)

In [36]:
dataset.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [37]:
dataset['Outcome'].value_counts()

0    500
1    268
Name: Outcome, dtype: int64

In [38]:
dataset.groupby('Outcome').mean()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Outcome,,,,,,,,
0,3.298000,109.980000,68.184000,19.664000,68.792000,30.304200,0.429734,31.190000
1,4.865672,141.257463,70.824627,22.164179,100.335821,35.142537,0.550500,37.067164


## Splitting the data set into the Training set and the Test set

In [39]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

## Feature Scaling

In [40]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Training the Model on the Training set

In [41]:
from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

## Model Evaluation

In [42]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score

### Accuracy of the Training data

In [43]:
y_pred_train = classifier.predict(X_train)
training_data_accuracy = accuracy_score(y_pred_train, y_train)

print(confusion_matrix(y_pred_train, y_train))
print('Accuracy score of the training data = {:.2f} %'.format(training_data_accuracy * 100))

[[370   0]
 [  0 206]]
Accuracy score of the training data = 100.00 %


### Accuracy of the Test set

In [44]:
y_pred_test = classifier.predict(X_test)
test_data_accuracy = accuracy_score(y_pred_test, y_test)

print(confusion_matrix(y_pred_test, y_test))
print('Accuracy of the test data = {:.2f} %'.format(test_data_accuracy * 100))

[[107  23]
 [ 23  39]]
Accuracy of the test data = 76.04 %


### K-Fold Cross Validation on the Training set

In [45]:
accuracies = cross_val_score(estimator = classifier,
                             X = X_train,
                             y = y_train,
                             cv = 10)

print('Accuracy = {:.2f} %'.format(accuracies.mean() * 100))
print('Standard Deviation = {:.2f} %'.format(accuracies.std() * 100))

Accuracy = 71.86 %
Standard Deviation = 5.95 %


### K-Fold Cross Validation on the Test set

In [46]:
accuracies = cross_val_score(estimator = classifier,
                             X = X_test,
                             y = y_test,
                             cv = 10)

print('Accuracy = {:.2f} %'.format(accuracies.mean() * 100))
print('Standard Deviation = {:.2f} %'.format(accuracies.std() * 100))

Accuracy = 75.47 %
Standard Deviation = 8.89 %
